# Spectral line selection

This notebook allows the user to display small windows of a spectrum datafile to select with mouse clicks the observed spectral lines. The notebook stores the frequency and corresponding Y-axis value of the peak in an output .csv file.
An optional telescope information file can be selected to convert antenna temperature to main beam temperature. This has been tested with Yebes 40m data, other telescopes may need additional code adjustments to correctly read telescope information.

* Left mouse button to select points
* Middle mouse buttom to delete previous point
* Right mouse button to go to the next window

Input variables can be modified to select different parts of the spectrum and range in MHz of the window.

User input:

In [ ]:
# Data files
spectrum_file = "/home/david/Documents/MasterAstro/practicas-master/Data/G10-47_all_reduced.fits"
telescope_info_file = "/home/david/Documents/MasterAstro/practicas-master/Data/Yebes_40m_W_band.csv"
# Use main beam temperature or antenna temperature
use_TMB = True
# Input parameters
freq_start = 85990
freq_stop = 86500
window_size = 30    # in MHz
# Output file
output_file_name = "selected_lines.csv"

Import libraries.

In [ ]:
# Import libraries

import astropy.constants as const
from astropy.io import fits
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats
from astropy.table import Table, QTable
import astropy.units as u
from astropy.wcs import WCS
import matplotlib as mpl
import matplotlib.axes as maxes
import matplotlib.patches as patches
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from pathlib import Path
import scipy.interpolate

from prac_utils import find_nearest_idx
from prac_utils import x2freq


Read the spectrum data file and beam efficiency from telescope info

In [ ]:
# Read full spectra
ccd = CCDData.read(spectrum_file)

# Read telescope data obtained from its webpage
if use_TMB:
    telescope_info = QTable.read(telescope_info_file, format='ascii')
    telescope_info['freq'] = telescope_info['freq'] * 1e3 * u.MHz
    telescope_info['lambda'].unit = u.mm
    telescope_info['theta'].unit = u.arcsec
    telescope_info['Jy/K'].unit = u.Jy / u.K

Create interpolation data of the telescope.

In [ ]:
# Create interpolation and extrapolation for the beam efficiency

if use_TMB:
    pol = 'V'
    x = telescope_info['freq'][telescope_info['polarization']==pol]
    y = telescope_info['eta_MB'][telescope_info['polarization']==pol]
    interp_band_W = scipy.interpolate.interp1d(x, y, fill_value='extrapolate')

Use tkinter backend for matplotlib

In [ ]:
%matplotlib tk

In the next cellblock:
* yroi is a list with one tuple inside
* yrois is a list of tuples

In [ ]:
# Data
y_axis = ccd.data[0, 0, 0, :]    # need to check slicing in header
freq_axis = x2freq(ccd=ccd) * 1e-6    # convert to MHz
if ccd.header['BUNIT'] == 'K':
    if use_TMB:
        etas = interp_band_W(freq_axis)
        T_beam = y_axis / etas
        y = T_beam
    else:
        y = y_axis
else:
    y = y_axis
freq_windows = np.arange(freq_start, freq_stop, window_size)

# Main loop
yrois = np.array([])
for i in range(len(freq_windows)):
    fig, ax, = plt.subplots()
    ax.plot(freq_axis, y)

    # set figure x limits
    xlim = (freq_windows[i], freq_windows[i]+30)
    ax.set_xlim(xlim)
    # set figure y limits
    min_idx = find_nearest_idx(freq_axis, xlim[0])
    max_idx = find_nearest_idx(freq_axis, xlim[1])
    min_y = min(y[min_idx:max_idx])
    max_y = max(y[min_idx:max_idx])
    ylim = (min_y - 0.15*(max_y-min_y), max_y + 0.2*(max_y-min_y))
    ax.set_ylim(ylim)

    # figure design
    ax.set_xlabel('Freq (MHz)')
    if ccd.header['BUNIT'] == 'K':
        if use_TMB:
            ax.set_ylabel(r'$T_{\rm MB}$ (K)')
        else:
            ax.set_ylabel(r'$T_{\rm A}^{*}$ (K)')
    else:
        ax.set_ylabel('Y axis')
    #plt.show()
    yroi = plt.ginput(n=0,timeout=0, mouse_add=1, mouse_pop=2, mouse_stop=3)
    if yroi != []:
        yrois = np.append(yrois, yroi)

    plt.close()

Save the data in a table

In [ ]:
line_clicks = np.reshape(yrois, [int(len(yrois)/2), 2])    # separate each click in its own dimension
line_points = np.transpose(line_clicks)    # transpose to hace two dimensions: freqs and T_antenna
selected_lines = Table(data=[line_points[0], line_points[1]], names=['Freq(MHz)', 'T_Beam(K)'])
selected_lines.write(output_file_name, overwrite=True)
selected_lines